## Import Libraries

In [23]:
import os
import random
import time
import datetime

from openai import OpenAI

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  
import wandb
from wandb.sdk.data_types.trace_tree import Trace

from langchain.agents import AgentType, initialize_agent
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool

from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

In [3]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

openai_api_key = os.getenv("OPENAI_API_KEY")

## Setup

In [4]:
client = OpenAI()

In [5]:
PROJECT = "llm_evaluation"
MODEL_NAME = "gpt-3.5-turbo"

In [37]:
# wandb.login(anonymous="allow")

In [38]:
# run = wandb.init(project=PROJECT, job_type="generation")

## Simple Generations

In [8]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

In [9]:
def generate_and_print(system_prompt, user_prompt, table, n=5):
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    start_time = time.time()
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
    )
    
    elapsed_time = time.time() - start_time
    for response in responses.choices:
        generation = response.message.content
        print(generation)
        
    table.add_data(
        system_prompt,
        user_prompt,
        [response.message.content for response in responses.choices],
        elapsed_time,
        datetime.datetime.fromtimestamp(responses.created),
        responses.model,
        responses.usage.prompt_tokens,
        responses.usage.completion_tokens,
        responses.usage.total_tokens
    )

In [10]:
system_prompt = """You are a creative copywriter.
You're given a category of game asset, \
and your goal is to design a name of that asset.
The game is set in a fantasy world \
where everyone laughs and respects each other, 
while celebrating diversity."""

In [11]:
# Define W&B Table to store generations
columns = [
    "system_prompt", "user_prompt", "generations", "elapsed_time", "timestamp",
    "model", "prompt_tokens", "completion_tokens", "total_tokens"
]

table = wandb.Table(columns=columns)

In [12]:
user_prompt = "hero"
generate_and_print(system_prompt, user_prompt, table)

Harmony Champion
Champion of Unity
Champion of Harmony
Champion of Harmony
Unity Champion


In [13]:
user_prompt = "jewel"
generate_and_print(system_prompt, user_prompt, table)

Joyful Gemstones
Sparkling Harmony Gems
 Harmony Gemstones
Whimsy Gemstones
DazzleGleam Gems


In [41]:
# wandb.log({"simple_generations": table})
# run.finish()

## Using Tracer to log more complex chains

Let's design an LLM chain that will first randomly pick a fantasy world, and then generate character names. We will demonstrate how to use Tracer in such scenario. We will log the inputs and outputs, start and end times, whether the OpenAI call was successful, the token usage, and additional metadata.

In [15]:
worlds = [
    "a mystic medieval island inhabited by intelligent and funny frogs",
    "a modern castle sitting on top of a volcano in a faraway galaxy",
    "a digital world inhabited by friendly machine learning engineers"
]

### Define your config

In [16]:
model_name = "gpt-3.5-turbo"
temperature = 0.7

system_message = """You are a creative copywriter. 
You're given a category of game asset and a fantasy world.
Your goal is to design a name of that asset.
Provide the resulting name only, no additional description.
Single name, max 3 words output, remember!"""

In [17]:
def run_creative_chain(query):
    # part 1 - a chain is started...
    start_time_ms = round(datetime.datetime.now().timestamp() * 1000)

    root_span = Trace(
          name="MyCreativeChain",
          kind="chain",
          start_time_ms=start_time_ms,
          metadata={"user": "student_1"},
          model_dict={"_kind": "CreativeChain"}
          )

    # part 2 - your chain picks a fantasy world
    time.sleep(3)
    world = random.choice(worlds)
    expanded_prompt = f'Game asset category: {query}; fantasy world description: {world}'
    tool_end_time_ms = round(datetime.datetime.now().timestamp() * 1000)

    # create a Tool span 
    tool_span = Trace(
          name="WorldPicker",
          kind="tool",
          status_code="success",
          start_time_ms=start_time_ms,
          end_time_ms=tool_end_time_ms,
          inputs={"input": query},
          outputs={"result": expanded_prompt},
          model_dict={"_kind": "tool", "num_worlds": len(worlds)}
          )

    # add the TOOL span as a child of the root
    root_span.add_child(tool_span)

    # part 3 - the LLMChain calls an OpenAI LLM...
    messages=[
      {"role": "system", "content": system_message},
      {"role": "user", "content": expanded_prompt}
    ]

    responses = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=temperature,
        max_tokens=12
    )

    llm_end_time_ms = round(datetime.datetime.now().timestamp() * 1000)
    
    for response in responses.choices:
        response_text = response.message.content
        
    token_usage = responses.usage.total_tokens

    llm_span = Trace(
          name="OpenAI",
          kind="llm",
          status_code="success",
          metadata={"temperature":temperature,
                    "token_usage": token_usage, 
                    "model_name":model_name},
          start_time_ms=tool_end_time_ms,
          end_time_ms=llm_end_time_ms,
          inputs={"system_prompt":system_message, "query":expanded_prompt},
          outputs={"response": response_text},
          model_dict={"_kind": "Openai", "engine": responses.model, "model": responses.object}
          )

    # add the LLM span as a child of the Chain span...
    root_span.add_child(llm_span)

    # update the end time of the Chain span
    root_span.add_inputs_and_outputs(
          inputs={"query":query},
          outputs={"response": response_text})

    # update the Chain span's end time
    root_span.end_time_ms = llm_end_time_ms


    # part 4 - log all spans to W&B by logging the root span
    root_span.log(name="creative_trace")
    print(f"Result: {response_text}")

In [42]:
# wandb.init(project=PROJECT, job_type="generation")

In [19]:
run_creative_chain("hero")

Result: Volcano Knight


In [20]:
run_creative_chain("jewel")

Result: Volcanite Gem


In [43]:
# wandb.finish()

## Langchain Agent

In the third scenario, we'll introduce an agent that will use tools such as WorldPicker and NameValidator to come up with the ultimate name

In [44]:
# wandb.init(project=PROJECT, job_type="generation")

In [25]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

In [26]:
llm = ChatOpenAI()

In [29]:
class WorldPickerTool(BaseTool):
    name:str = "pick_world"
    description:str = "pick a virtual game world for your character or item naming"
    worlds:list[str] = [
        "a mystic medieval island inhabited by intelligent and funny frogs",
        "a modern anthill featuring a cyber-ant queen and her cyber-ant-workers",
        "a digital world inhabited by friendly machine learning engineers"
    ]

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        time.sleep(1)
        return random.choice(self.worlds)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("pick_world does not support async")

In [30]:
class NameValidatorTool(BaseTool):
    name:str = "validate_name"
    description:str = "validate if the name is properly generated"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        time.sleep(1)
        if len(query) < 20:
            return f"This is a correct name: {query}"
        else:
            return f"This name is too long. It should be shorter than 20 characters."

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("validate_name does not support async")

In [32]:
tools = [WorldPickerTool(), NameValidatorTool()]

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

In [34]:
agent.invoke(
    "Find a virtual game world for me and imagine the name of a hero in that world"
)



> Entering new AgentExecutor chain...
I need to pick a virtual game world first and then come up with a hero name within that world.
Action: pick_world
Action Input: Netherworld
Observation: a digital world inhabited by friendly machine learning engineers
Thought:Now I need to come up with a hero name within the Netherworld.
Action: validate_name
Action Input: Shadowstrike
Observation: This is a correct name: Shadowstrike
Thought:I now know the final answer
Final Answer: The hero name in the virtual game world of Netherworld is Shadowstrike.

> Finished chain.


{'input': 'Find a virtual game world for me and imagine the name of a hero in that world',
 'output': 'The hero name in the virtual game world of Netherworld is Shadowstrike.'}

In [35]:
agent.invoke(
    "Find a virtual game world for me and imagine the name of food in that world."
)



> Entering new AgentExecutor chain...
I need to pick a virtual game world and come up with a food name for that world.
Action: pick_world
Action Input:
Observation: a digital world inhabited by friendly machine learning engineers
Thought:I need to come up with a food name that fits in a world inhabited by friendly machine learning engineers.
Action: validate_name
Action Input: Crunchy Circuits
Observation: This is a correct name: Crunchy Circuits
Thought:I now know the final answer
Final Answer: The name of the food in the virtual game world is Crunchy Circuits.

> Finished chain.


{'input': 'Find a virtual game world for me and imagine the name of food in that world.',
 'output': 'The name of the food in the virtual game world is Crunchy Circuits.'}

In [45]:
# wandb.finish()